In [92]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from scipy.sparse import csr_matrix
import numpy.matlib


two_moon=datasets.make_moons(n_samples=10,noise=0.1)
X3=two_moon[0]
KM3=KMeans(n_clusters=3,init='random',random_state=5)
KM3.fit(X3)
index=KM3.predict(X3)
#plt.scatter(X3[:,0],X3[:,1],c=list(index))

# KMM_Function
## **input:**
- x : Data matrix
- c : The number of cluster
- m : The number of subcluster
- k : The number of neighbor prototype
    
## **output:**
- lakmm : The <font color='red'>cluster</font> assignment for each point
- lamm : The <font color='red'>sub-cluster</font> assignment for each point
- bigaph : Bipartite graph (Data point and neighbor prototype)
- A : Coordinates of the multiple means 

    

In [47]:
import numpy as np

class Clustering(object):
    
    def __init__(self): pass
    
    def k_multiple_means(x,c,m,k=5):
        
        iterTimes=1000             #迭代次數
        n=np.size(x,axis=0)        #共有幾筆資料
        dim=np.size(x,axis=1)      #每筆資料的維度
        
        if m < 6:                  # k 值必須小於等於 5
            k=c-1
    
        
        A = np.random.random([m,dim])    #亂數產生k個dim維度的值 ----------- (initial) The prototype matrix 
            
        #S：connection probability matrix(initial)
        S=np.array(np.ones([n,m]))   
        
        #--------用k-means找出初始index--------
            
        KM=KMeans(n_clusters=m,init='random',random_state=6)
        KM.fit(x)
        A_index=list(KM.predict(x))                            #A_index : 紀錄 Data point 屬於哪個 Prototype
        
        for count_iter in range(iterTimes):               
            
            old_A=np.copy(A)  
            A=meanIdex(x,A_index,m,S)    
             
            #--------判斷收斂--------
            th=np.linalg.norm(A-old_A)                       #計算兩個prototype的距離

            if th<np.spacing(1):                             #電腦區分兩個數的差的最小精度，當兩個數的差小於這個值，則電腦認為兩個數相等
                break
        
        A=meanIdex(x,A_index,m,S)
        #--------做完約束的 mm ，就是 A_index
        
        return A,A_index

# 測試視窗

In [204]:
A=Clustering.k_multiple_means(X3,3,6,1)
k=5
n=np.size(X3,axis=0)     #計算 Data point 數量
m=6     #計算 Prototype 數量
dist=np.array(np.zeros([n,m]))    #初始化距離矩陣
z=np.copy(dist)

for i in range(m):
    c=A[0][i]
    dist[:,i] = np.linalg.norm(X3-c,axis=1)              #計算每個 Data Point 與 每個 Prototype 的距離
    
disXt = np.copy(dist)                                    #會修改到 dist 的值，先做備份

k_dist=np.array(np.zeros([n,k+1]))                       #（初始化 0）生成 n 個 Data Point * (k+1)鄰近點矩陣
k_index=np.array(np.zeros([n,k])) 
tmp = np.array(np.zeros([n,k]))

for i in range(k+1):
    
#     print(disXt)
#     print(k_dist) 
#     print(k_index)
    
    k_dist[:,i]=disXt.min(axis=1)
    if i!=k:                                             #對照 id(:,end) = [];
        k_index[:,i]=disXt.argmin(axis=1)
        for j in range(n):
            a = k_index[j][i]
            disXt[j][int(a)]=1000000000000000

y=0

sum_matrix = np.sum(k_dist,axis=1)         

for i in range(n): #計算 k+1 之前的總和
    
    sum_matrix[i]-=k_dist[i][k]


for i in range(n): #計算 y 值
    
    y=y+((k*k_dist[i][k])-sum_matrix[i])                 

y=0.5*y/n

for i in range(n):
    
    for j in range(k):
        
        tmp[i][j]=(k_dist[i][k]-k_dist[i][j])/((k*k_dist[i][k])-sum_matrix[i])      #算出與鄰近prototype連接機率

count=[]

count=np.matlib.repmat(np.arange(n), 1, k)
k_index=k_index.reshape(1,n*k)
tmp=tmp.reshape(1,n*k)

#count=np.array(count)
#k_index=k_index.astype(int)

print(count[0])
print(k_index[0])
print(tmp[0])
z = csr_matrix((tmp[0], (count[0], k_index[0])),shape=(n, m))
print(np.array(z))



[0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6
 7 8 9 0 1 2 3 4 5 6 7 8 9]
[4. 2. 3. 1. 5. 3. 4. 1. 5. 2. 1. 3. 4. 2. 5. 2. 0. 4. 1. 3. 0. 2. 1. 4.
 3. 1. 4. 3. 2. 0. 2. 4. 0. 1. 3. 0. 2. 4. 1. 3. 3. 1. 5. 4. 2. 5. 3. 4.
 1. 2.]
[0.47905813 0.2042292  0.16717648 0.13176963 0.01776655 0.31363209
 0.22999634 0.22058843 0.14721375 0.0885694  0.44874577 0.32424678
 0.16578724 0.04431209 0.01690812 0.29495926 0.25274118 0.20057542
 0.15662103 0.0951031  0.32718965 0.24261674 0.17375994 0.16166793
 0.09476574 0.29840927 0.20362205 0.17633734 0.17588456 0.14574678
 0.34492437 0.24140291 0.22294993 0.11323721 0.07748557 0.30184537
 0.28482108 0.18177765 0.14682877 0.08472713 0.32966356 0.21619883
 0.20903466 0.18046043 0.06464252 0.36923764 0.23423136 0.18727737
 0.12964472 0.07960893]
  (0, 0)	0.3271896505572755
  (0, 1)	0.44874576833821894
  (0, 2)	0.34492437356980826
  (0, 3)	0.3296635622161548
  (0, 4)	0.4790581265485693
  (1, 1)	0.2161988280280085
  (1, 2)	0.44

---
## Function "<font color='red'>meanIdex</font>" 

### 功能：更新 <font color='red'>A</font> (Prototype 座標 ) ----------> 對應公式（5）

#### **(input)**
- **x**       : Data Point<br>
- **A_index** : 紀錄 Data point 屬於哪個 Prototype (Label值)<br>
- **m**       : sub-cluster(prototype)數量<br>
- **S**       : 連接機率矩陣<br>
    
#### **(return)**
- 更新後的矩陣 **A** (Prototype 座標)<br>

In [35]:
def meanIdex(x,A_index,m,S):

    A=np.zeros([m,2])
    n=np.size(x,axis=0)
    
    for j in range(m):
            index=0               #為了符合先乘除後加減，判斷使用
            mu=0
            su=0
            for i in range(n):       
                if A_index[i]==j:
                    
                    if index==0:
                        mu=mu+x[i]
                        mu=mu*S[i][j]
                        index=1
                    else:
                        mu=mu+(x[i]*S[i][j])
                    
                    su=su+S[i][j]
                    
            A[j]=mu/su       
    
    return A

In [36]:
def CLR(x,c,A,k,y,l):
    
    # ----- Parameter -----
    # x : Data matrix
    # c : The number of cluster
    # A : The prototype matrix
    # k : The number of neighbor points
    
    n=np.size(x,axis=0)
    m=np.size(A,axis=0)
    
    

---
## Function "<font color='red'>ConstructA_NP</font>" 

### 功能： <font color='red'>///</font> 

#### **(input)**
- **x**       : Data Point<br>
- **A**       : Prototype 座標矩陣<br>
- **k**       : 鄰近Prototype的數量<br>
- **isSparse**       :  <br>
    
#### **(return)**
- **z** : (稀疏矩陣)儲存鄰近 Prototype 的機率<br>
- **y** : (gamma)算平均值，當鄰近 Prototype 的距離超過平均值則不連接<br>
- **dist** : 計算每個 Data Point 與 每個 Prototype 的距離
- **k_index**

In [37]:
def ConstructA_NP(x,A,k=5,isSparse=1):
    
    # k 不能大於 sub_cluster 的數量
    
    n=np.size(x,axis=0)     #計算 Data point 數量
    m=np.size(A,axis=0)     #計算 Prototype 數量
    dist=np.zeros([n,m])    #初始化距離矩陣
    
    for i in range(m):
        c=A[i]
        dist[:,i] = np.linalg.norm(x-c,axis=1)     #計算每個 Data Point 與 每個 Prototype 的距離

    
    
        
    disXt = np.copy(dist)                                    #會修改到 dist 的值，先做備份
    
    k_dist = np.zeros([n,k+1])                               #(初始化 0）生成 n 個 Data Point * (k+1)鄰近點矩陣
    k_index = np.zeros([n,k])                                #這裡不能跟 matlab一樣寫成 k_index = k_dist 會變成指向同一個記憶體造成覆蓋
    tmp = np.array(np.zeros([n,k]))
   
    for i in range(k+1):
    
        k_dist[:,i]=disXt.min(axis=1)
        if i!=k:                                             #對照 id(:,end) = [];
            k_index[:,i]=disXt.argmin(axis=1)
            for j in range(n):
                a = k_index[j][i]
                disXt[j][int(a)]=1000000000000000

    y=0

    sum_matrix = np.sum(k_dist,axis=1)         

    for i in range(n): #計算 k+1 之前的總和
    
        sum_matrix[i]-=k_dist[i][k]


    for i in range(n): #計算 y 值
    
        y=y+((k*k_dist[i][k])-sum_matrix[i])                 

    y=0.5*y/n

    for i in range(n):
    
        for j in range(k):
        
            tmp[i][j]=(k_dist[i][k]-k_dist[i][j])/((k*k_dist[i][k])-sum_matrix[i])      #算出與鄰近prototype連接機率

    count=[]

    count=np.matlib.repmat(np.arange(n), 1, k)
    k_index=k_index.reshape(1,n*k)
    tmp=tmp.reshape(1,n*k)

    z = csr_matrix((tmp[0], (count[0], k_index[0])),shape=(n, m))


In [38]:
dist=np.zeros(2,3)
temp=1
dist(temp)=5
print(dist)

SyntaxError: can't assign to function call (<ipython-input-38-141e20c1fc85>, line 3)